In [20]:
import pandas as pd
books = pd.read_csv("books_cleaned.csv")

In [21]:
category_mapping = {'Fiction' : "Fiction",
 'Juvenile Fiction': "Children's Fiction",
 'Biography & Autobiography': "Nonfiction",
 'History': "Nonfiction",
 'Literary Criticism': "Nonfiction",
 'Philosophy': "Nonfiction",
 'Religion': "Nonfiction",
 'Comics & Graphic Novels': "Fiction",
 'Drama': "Fiction",
 'Juvenile Nonfiction': "Children's Nonfiction",
 'Science': "Nonfiction",
 'Poetry': "Fiction"}

books["simple_categories"] = books["categories"].map(category_mapping)

In [22]:
from transformers import pipeline
fiction_categories = ['Fiction', 'Nonfiction']
pipe = pipeline('zero-shot-classification', model = "facebook/bart-large-mnli", device=0)


Device set to use cpu


In [27]:
sequence = books.loc[books['simple_categories'] == "Fiction", "description"].reset_index(drop=True)[3]

In [28]:
pipe(sequence, fiction_categories)

{'sequence': 'A Few Weeks After Marrying An Attractive Young Widow, Gordon Cloade Is Tragically Killed By A Bomb Blast In The London Blitz. Overnight, The Former Mrs Underhay Finds Herself In Sole Possession Of The Cloade Family Fortune. Shortly Afterwards, Hercule Poirot Receives A Visit From The Dead Man S Sister-In-Law Who Claims She Has Been Warned By Spirits That Mrs Underhay S First Husband Is Still Alive. Poirot Has His Suspicions When He Is Asked To Find A Missing Person Guided Only By The Spirit World. Yet What Mystifies Poirot Most Is The Woman S True Motive For Approaching Him&',
 'labels': ['Nonfiction', 'Fiction'],
 'scores': [0.6068205237388611, 0.39317941665649414]}

In [29]:
import numpy as np
max_index = np.argmax(pipe(sequence, fiction_categories)["scores"])
max_label = pipe(sequence,fiction_categories)["labels"][max_index]
max_label

'Nonfiction'

In [31]:
def generate_predictions(sequence, categories):
  predictions = pipe(sequence, categories)["scores"]
  max_index = np.argmax(predictions)
  max_label = pipe(sequence, categories)["labels"][max_index]
  return max_label